# Supporting a custom L0 RISC-V accelerator


Let's assume we have been given the following from God:

 - A program compiled half-way down to RISC-V Assembly
 - The format of this program is what we call "RISC-V SSA"

Let's not worry too much about what all this means, and instead just look at it:


In [11]:
from riscv.riscv_ssa import *
from xdsl.dialects.builtin import ModuleOp
from riscv.emulator_iop import run_riscv, print_riscv_ssa

p = Printer(target=Printer.Target.MLIR)

## Our SSA RISC-V:

In [12]:
module = ModuleOp.from_region_or_ops([
    LabelOp.get('main'),
    a0  := LIOp.get(82),
    a1  := LIOp.get(5),
    mul := MULOp.get(a0, a1),
    a2  := LIOp.get(10),
    add := AddOp.get(mul, a2),
    ECALLOp.get(93, add)
])

**let's print it out:**

In [3]:
p.print(module)

"builtin.module"() ({
  "riscv_ssa.label"() {"label" = #riscv.label<main>} : () -> ()
  %0 = "riscv_ssa.li"() {"immediate" = 82 : i32} : () -> #riscv_ssa.reg
  %1 = "riscv_ssa.li"() {"immediate" = 5 : i32} : () -> #riscv_ssa.reg
  %2 = "riscv_ssa.mul"(%0, %1) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
  %3 = "riscv_ssa.li"() {"immediate" = 10 : i32} : () -> #riscv_ssa.reg
  %4 = "riscv_ssa.add"(%2, %3) : (#riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
  "riscv_ssa.ecall"(%4) {"syscall_num" = 93 : i32} : (#riscv_ssa.reg) -> ()
}) : () -> ()


This *really* doesn't look like RISC-V. But if I look at it for a while, I can see some familiar stuff:


```
    %0 = "riscv_ssa.li"() {"immediate" = 82 : i32}
    ^^              ^^                   ^^
    Result?         Op-name              Argument
```

We seem to have the `li` pseudo-op here, with an immediate value of `82`. It's result is stored in `%0`.

## A short (and incomplete) introduction to SSA

 - We have infinitely many variables (or registers)
 - We can only assign to each register once
 
This Notebook provides a function to print the RISC-V SSA in a more RISC-V-like format:

In [14]:
print(print_riscv_ssa(module))

.text
main:
	li	%0, 82
	li	%1, 5
	mul	%2, %0, %1
	li	%3, 10
	add	%4, %2, %3
	mv	a0, %4
	li	a7, 93
	scall



Okay, this isn't exactly valid RISC-V, but it's relatively close. We can try to run it in an RISC-V Emulator.

Luckily, this Notebook comes with a built-in emulator:

In [17]:
run_riscv(print_riscv_ssa(module), extensions=(RV_fmadd,))

[CPU] Started running from example.asm:.text at main (0x100) + 0x0
Program(name=example.asm,sections=set(),base=['.text'])
   Running 0x00000100: li %0, 82
Invalid register: %0


**Aaaargh, this is bad! We need to do register allocation!**


If *only* there was a way to make the emulator accept our "unlimited register" RISC-V...


In [18]:
run_riscv(print_riscv_ssa(module), extensions=(RV_fmadd,), unlimited_regs=True)

[CPU] Started running from example.asm:.text at main (0x100) + 0x0
Program(name=example.asm,sections=set(),base=['.text'])
   Running 0x00000100: li %0, 82
   Running 0x00000104: li %1, 5
   Running 0x00000108: mul %2, %0, %1
   Running 0x0000010C: li %3, 10
   Running 0x00000110: add %4, %2, %3
   Running 0x00000114: mv a0, %4
   Running 0x00000118: li a7, 93
   Running 0x0000011C: scall 
[CPU] Program exited with code 420


## Let's Accelerate This:

let's start simple. We want to add a fused multiply-add instruction to our RISC-V ISA.

Let's define it's structure as `fmadd  rd, rs1, rs2, rs3`

We first need to tell our compiler about the structure of our new instruction. For that we can use `xDSL` and it's interface for defining new Operations called `irdl`:

In [4]:
@irdl_op_definition
class FmaddOp(Operation):
    name = "riscv_ssa.fmadd"
    
    rd: Annotated[OpResult, RegisterType]
    """
    We return a single value in a register
    """
    
    rs1: Annotated[Operand, RegisterType]
    rs2: Annotated[Operand, RegisterType]
    rs3: Annotated[Operand, RegisterType]
    """
    We take three arguments (Operands), which are also registers.
    """
    
    @classmethod
    def get(cls, *rs):
        return cls.build(operands=rs, result_types=[RegisterType()])

In [5]:
from xdsl.pattern_rewriter import (GreedyRewritePatternApplier,
                                   PatternRewriter, PatternRewriteWalker,
                                   RewritePattern, op_type_rewrite_pattern)

class FmaddOpOptimizer(RewritePattern):
    @op_type_rewrite_pattern
    def match_and_rewrite(self, add: AddOp, rewriter: PatternRewriter):        
        for operand in add.operands:
            if isinstance(operand.op, MULOp):
                other_operand = add.rs1 if operand == add.rs2 else add.rs2                
                a, b = operand.op.operands

                rewriter.replace_matched_op(
                    FmaddOp.get(a, b, other_operand)
                )

                rewriter.erase_op(operand.op)
                break

In [6]:
PatternRewriteWalker(GreedyRewritePatternApplier([FmaddOpOptimizer()])).rewrite_module(module)

In [7]:
p.print(module)

"builtin.module"() ({
  "riscv_ssa.label"() {"label" = #riscv.label<main>} : () -> ()
  %0 = "riscv_ssa.li"() {"immediate" = 82 : i32} : () -> #riscv_ssa.reg
  %1 = "riscv_ssa.li"() {"immediate" = 5 : i32} : () -> #riscv_ssa.reg
  %3 = "riscv_ssa.li"() {"immediate" = 10 : i32} : () -> #riscv_ssa.reg
  %5 = "riscv_ssa.fmadd"(%0, %1, %3) : (#riscv_ssa.reg, #riscv_ssa.reg, #riscv_ssa.reg) -> #riscv_ssa.reg
  "riscv_ssa.ecall"(%5) {"syscall_num" = 93 : i32} : (#riscv_ssa.reg) -> ()
}) : () -> ()


In [8]:
print(print_riscv_ssa(module))

.text
main:
	li	%0, 82
	li	%1, 5
	li	%2, 10
	fmadd	%3, %0, %1, %2
	mv	a0, %3
	li	a7, 93
	scall



In [9]:
from riscemu.instructions import InstructionSet
from riscemu.types import Instruction

class RV_fmadd(InstructionSet):
    def instruction_fmadd(self, ins: Instruction):
        rd, rs1, rs2, rs3 = (ins.get_reg(i) for i in (0,1,2,3))
        
        self.regs.set(
            rd, (self.regs.get(rs1) * self.regs.get(rs2)) + self.regs.get(rs3)
        )

In [10]:
run_riscv(print_riscv_ssa(module), extensions=(RV_fmadd,), unlimited_regs=True)

[CPU] Started running from example.asm:.text at main (0x100) + 0x0
Program(name=example.asm,sections=set(),base=['.text'])
   Running 0x00000100: li %0, 82
   Running 0x00000104: li %1, 5
   Running 0x00000108: li %2, 10
   Running 0x0000010C: fmadd %3, %0, %1, %2
   Running 0x00000110: mv a0, %3
   Running 0x00000114: li a7, 93
   Running 0x00000118: scall 
[CPU] Program exited with code 420
